In [104]:
#V:1.1 - Coded by Alexandre Ménard - 2024
import ipywidgets as W
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from numpy import exp, sqrt, log
from scipy import stats
from math import *

#Global var
type_opt = ""
spot = 0
strike = 0
volatility = 0
interest_rate = 0
maturity = 0
dividend = 0
d1 = 0
d2 = 0
d1RC = 0
delta = 0
gamma = 0
vega = 0
theta = 0
rho = 0

def european_opt_pricing(b):
    output.clear_output()
    if (call_checkbox.value == True and put_checkbox.value == False) or (call_checkbox.value == False and put_checkbox.value == True):
        global type_opt
        if call_checkbox.value == True and put_checkbox.value == False:
            type_opt = 'Call'
        elif call_checkbox.value == False and put_checkbox.value == True:
            type_opt = 'Put'

        global spot
        global strike
        global volatility
        global interest_rate
        global maturity
        global dividend
        spot = float(stock_price_box.value)
        strike = float(strike_price_box.value)
        volatility = float(volatility_box.value)
        interest_rate = float(interest_rate_box.value)
        maturity = float(maturity_box.value)
        dividend = float(dividend_rate_box.value)

        #Greeks (1st order)
        delta_opt(spot)
        gamma_opt(spot)
        vega_opt(spot)
        theta_opt(spot)
        rho_opt(spot)
    
        if type_opt == 'Call':
            price_opt = round(spot*stats.norm.cdf(d1)*exp(-dividend*maturity)-strike*(exp(-interest_rate*maturity))*stats.norm.cdf(d2),2)
        elif type_opt == 'Put':
            price_opt = round(strike*(exp(-interest_rate*maturity))*stats.norm.cdf(-d2)-spot*stats.norm.cdf(-d1)*exp(-dividend*maturity),2)
        
        #if type_opt == 'Call':
            #rho = round((strike*maturity*exp(-interest_rate*maturity)*stats.norm.cdf(d2))/100,4)
        #elif type_opt == 'Put':
            #rho = round((-strike*maturity*exp(-interest_rate*maturity)*stats.norm.cdf(-d2))/100,4)
            
        #Results
        with output:
            print("Param : " + str(spot) + "," + str(strike) + "," + str(volatility) + "," + str(interest_rate) + "," + str(maturity) + "," + str(dividend))
            print("D1 : " + str(d1))
            print("D2 : " + str(d2))
            
        Price_box.value = str(price_opt)
        Delta_box.value = str(round(delta,4))
        Gamma_box.value = str(round(gamma,4))
        Vega_box.value = str(round(vega,4))
        Rho_box.value = str(round(rho,4))
        Theta_box.value = str(round(theta,5))
        
        with output:
            print("Pricing...")
            print("Final price : " + str(price_opt))
            delta_chart_button.disabled=False
            gamma_chart_button.disabled=False
            vega_chart_button.disabled=False
            theta_chart_button.disabled=False
            rho_chart_button.disabled=False
            #print("D1 prime : " + str(d1RC)) #For checking..
            #print(type(spot)) #For checking..
            #spot = np.linspace(1,200,100)
            #plt.plot(spot, delta_opt(spot,strike,volatility,interest_rate,maturity,dividend), color='red')
            #plt.xlim(0,200)
            #plt.ylim(
            #plt.plot([1, 2, 3, 4])
            #plt.ylabel('some numbers')
            #plt.show()
    else:
        with output:
            print("Please select call or put option, not both!")
            delta_chart_button.disabled=True
            gamma_chart_button.disabled=True
            vega_chart_button.disabled=True
            theta_chart_button.disabled=True
            rho_chart_button.disabled=True

def delta_chart(b):
    spot_temp = float(stock_price_box.value)
    output.clear_output()
    with output:
        spot_temp = np.linspace(spot_temp/2,spot_temp*1.75,100)
        plt.plot(spot_temp, delta_opt(spot_temp), color='red')
        plt.show()

def delta_opt(spot):
    global d1
    global d2
    global delta
    global d1RC
    if type_opt == 'Call':
        d1 = ((np.log(spot/strike)+((interest_rate-dividend)+(volatility**2)/2)*maturity)/(volatility*(maturity)))
        delta = stats.norm.cdf(d1)*exp(-dividend*maturity)
        d2 = d1-volatility*np.sqrt(maturity)
        d1RC = (1/(np.sqrt(2*pi)))*np.exp(-(d1**2)/2)
    elif type_opt == 'Put':
        d1 = ((np.log(spot/strike)+((interest_rate-dividend)+(volatility**2)/2)*maturity)/(volatility*(maturity)))
        delta = -(stats.norm.cdf(-d1)*exp(-dividend*maturity))
        d2 = d1-volatility*np.sqrt(maturity)
        d1RC = (1/(np.sqrt(2*pi)))*np.exp(-(d1**2)/2)
    return delta

def gamma_chart(b):
    spot_temp = float(stock_price_box.value)
    output.clear_output()
    with output:
        spot_temp = np.linspace(spot_temp/2,spot_temp*1.75,100)
        plt.plot(spot_temp, gamma_opt(spot_temp), color='red')
        plt.show()

def gamma_opt(spot):
    global gamma
    d1 = ((np.log(spot/strike)+((interest_rate-dividend)+(volatility**2)/2)*maturity)/(volatility*(maturity)))
    d1RC = (1/(np.sqrt(2*pi)))*np.exp(-(d1**2)/2)
    gamma = (d1RC*np.exp(-dividend*maturity))/(spot*volatility*np.sqrt(maturity))
    return gamma

def vega_chart(b):
    spot_temp = float(stock_price_box.value)
    output.clear_output()
    with output:
        spot_temp = np.linspace(spot_temp/2,spot_temp*1.75,100)
        plt.plot(spot_temp, vega_opt(spot_temp), color='red')
        plt.show()

def vega_opt(spot):
    global vega
    d1 = ((np.log(spot/strike)+((interest_rate-dividend)+(volatility**2)/2)*maturity)/(volatility*(maturity)))
    d1RC = (1/(np.sqrt(2*pi)))*np.exp(-(d1**2)/2)
    vega = (d1RC*np.exp(-dividend*maturity)*np.sqrt(maturity)*spot)/100
    return vega

def theta_chart(b):
    spot_temp = float(stock_price_box.value)
    output.clear_output()
    with output:
        spot_temp = np.linspace(1,spot_temp*2,100)
        plt.plot(spot_temp, theta_opt(spot_temp), color='red')
        plt.show()

def theta_opt(spot):
    global theta
    d1 = ((np.log(spot/strike)+((interest_rate-dividend)+(volatility**2)/2)*maturity)/(volatility*(maturity)))
    d2 = d1-volatility*np.sqrt(maturity)
    d1RC = (1/(np.sqrt(2*pi)))*np.exp(-(d1**2)/2)    
    if type_opt == 'Call':
        theta = (-((d1RC*(spot*np.exp(-dividend*maturity))*volatility)/(2*sqrt(maturity)))-interest_rate*(strike*exp(-interest_rate*maturity))*stats.norm.cdf(d2)+dividend*(spot*exp(-dividend*maturity))*stats.norm.cdf(d1))/252
    elif type_opt == 'Put':          
        theta = (-((d1RC*(spot*np.exp(-dividend*maturity))*volatility)/(2*sqrt(maturity)))+interest_rate*(strike*exp(-interest_rate*maturity))*stats.norm.cdf(-d2)-dividend*(spot*exp(-dividend*maturity))*stats.norm.cdf(-d1))/252
    return theta

def rho_chart(b):
    spot_temp = float(stock_price_box.value)
    output.clear_output()
    with output:
        spot_temp = np.linspace(1,spot_temp*2,100)
        plt.plot(spot_temp, rho_opt(spot_temp), color='red')
        plt.show()

def rho_opt(spot):
    global rho
    d1 = ((np.log(spot/strike)+((interest_rate-dividend)+(volatility**2)/2)*maturity)/(volatility*(maturity)))
    d2 = d1-volatility*np.sqrt(maturity)   
    if type_opt == 'Call':
        rho = (strike*maturity*exp(-interest_rate*maturity)*stats.norm.cdf(d2))/100
    elif type_opt == 'Put':
        rho = (-strike*maturity*exp(-interest_rate*maturity)*stats.norm.cdf(-d2))/100
    return rho
    
stock_price_box = W.Combobox(value='100', placeholder='100', description='Stock Price :')
strike_price_box = W.Combobox(value='100', placeholder='100', description='Strike Price :')
volatility_box = W.Combobox(value='0.2', placeholder='0.2', description='Volatility :')
interest_rate_box = W.Combobox(value='0.02', placeholder='0.03', description='Interest rate :')
maturity_box = W.Combobox(value='1', placeholder='1.5', description='Maturity :')
dividend_rate_box = W.Combobox(value='0.05', placeholder='0.05', description='Dividend yield :')
price_button = W.Button(description='Price !',disabled=False)
delta_chart_button = W.Button(description='Get Chart',disabled=True)
gamma_chart_button = W.Button(description='Get Chart',disabled=True)
theta_chart_button = W.Button(description='Get Chart',disabled=True)
vega_chart_button = W.Button(description='Get Chart',disabled=True)
rho_chart_button = W.Button(description='Get Chart',disabled=True)
output = W.Output()
call_checkbox = W.Checkbox(value=False,description='Call',disabled=False,indent=False)
put_checkbox = W.Checkbox(value=False,description='Put',disabled=False,indent=False)
Price_box = W.Text(value='',placeholder='Please price',description='Final Price:',disabled=True)
Delta_box = W.Text(value='',placeholder='Please price',description='Delta:',disabled=True)
Gamma_box = W.Text(value='',placeholder='Please price',description='Gamma:',disabled=True)
Theta_box = W.Text(value='',placeholder='Please price',description='Theta:',disabled=True)
Vega_box = W.Text(value='',placeholder='Please price',description='Vega:',disabled=True)
Rho_box = W.Text(value='',placeholder='Please price',description='Rho:',disabled=True)

type_opt = W.HBox([call_checkbox, put_checkbox])
line1 = W.HBox([stock_price_box, Price_box])
line2 = W.HBox([strike_price_box, Delta_box, delta_chart_button])
line3 = W.HBox([volatility_box, Gamma_box, gamma_chart_button])
line4 = W.HBox([interest_rate_box, Theta_box, theta_chart_button])
line5 = W.HBox([dividend_rate_box, Vega_box, vega_chart_button])
line6 = W.HBox([maturity_box, Rho_box, rho_chart_button])
#type_opt = W.HBox([W.Checkbox(value=False,description='Call',disabled=False,indent=False),W.Checkbox(value=False,description='Put',disabled=False,indent=False)])

display(type_opt)
display(line1)
display(line2)
display(line3)
display(line4)
display(line5)
display(line6)
display(price_button,output)
    
price_button.on_click(european_opt_pricing)
delta_chart_button.on_click(delta_chart)
gamma_chart_button.on_click(gamma_chart)
vega_chart_button.on_click(vega_chart)
theta_chart_button.on_click(theta_chart)
rho_chart_button.on_click(rho_chart)
    

Button(description='Price !', style=ButtonStyle())

Output()